In [1]:
%load_ext dotenv
%dotenv -o

import os
not not os.environ["POLYGON_API_KEY"]

True

In [2]:
import pyarrow as pa
import pyarrow.csv as pa_csv
from pyarrow import dataset as pa_ds
from pyarrow import compute as pa_compute
from pyarrow import fs as pa_fs
from pathlib import Path
import pandas as pd

import zipline_polygon_bundle as zpb

from zipline_polygon_bundle.config import PolygonConfig

from itables import show

In [3]:
config = PolygonConfig(
    environ=os.environ,
    calendar_name="XNYS",
    # start_session="2020-10-07",
    # end_session="2020-10-15",
    # start_session="2003-10-01",
    start_session="2016-01-05",
    # start_session="2023-01-03",
    end_session="2022-12-30",
    # end_session="2023-12-28",
    # end_session="2020-12-31",
    # end_session="2024-06-30",
)

In [4]:
trades_ds = zpb.trades_dataset(config)
trades_ds.schema

ticker: string not null
conditions: string not null
correction: int8 not null
exchange: int8 not null
id: string not null
participant_timestamp: int64 not null
price: double not null
sequence_number: int64 not null
sip_timestamp: int64 not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: int64 not null

In [25]:
# trades_ds.count_rows()
# 194543824321

194543824321

In [6]:
fragments = trades_ds.get_fragments()
fragment = next(fragments)
trades = fragment.to_table(schema=zpb.trades_schema(raw=True))
trades = zpb.cast_trades(trades)
trades

pyarrow.Table
ticker: string not null
conditions: string not null
correction: int8 not null
exchange: int8 not null
id: string not null
participant_timestamp: timestamp[ns, tz=UTC] not null
price: double not null
sequence_number: int64 not null
sip_timestamp: timestamp[ns, tz=UTC] not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: timestamp[ns, tz=UTC] not null
condition_values: list<item: uint8>
  child 0, item: uint8
----
ticker: [["A","A","A","A","A",...,"AAPL","AAPL","AAPL","AAPL","AAPL"],["AAPL","AAPL","AAPL","AAPL","AAPL",...,"ABS","ABS","ABS","ABS","ABS"],...,["YHOO","YHOO","YHOO","YHOO","YHOO",...,"ZICA","ZICA","ZICA","ZICA","ZICA"],["ZICA","ZICA","ZICA","ZICA","ZICA",...,"ZXZZT","ZXZZT","ZXZZT","ZXZZT","ZXZZT"]]
conditions: [["","","","","",...,"","","","",""],["","","","","",...,"","","","",""],...,["","","","","",...,"","","","",""],["","","","","",...,"12","15","15","12",""]]
correction: [[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,

In [8]:
trades.num_rows

5608283

In [16]:
df = trades.to_pandas()
df.info()
show(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5608283 entries, 0 to 5608282
Data columns (total 14 columns):
 #   Column                 Dtype              
---  ------                 -----              
 0   ticker                 object             
 1   conditions             object             
 2   correction             int8               
 3   exchange               int8               
 4   id                     object             
 5   participant_timestamp  datetime64[ns, UTC]
 6   price                  float64            
 7   sequence_number        int64              
 8   sip_timestamp          datetime64[ns, UTC]
 9   size                   int64              
 10  tape                   int8               
 11  trf_id                 int64              
 12  trf_timestamp          datetime64[ns, UTC]
 13  condition_values       object             
dtypes: datetime64[ns, UTC](3), float64(1), int64(3), int8(3), object(4)
memory usage: 486.7+ MB


In [14]:
df['condition_values'].value_counts()

condition_values
[0]         5279655
[3]          231358
[12]          56613
[2]           19406
[32]           9276
[15]           6280
[22]           5077
[20]            303
[4]              91
[7]              85
[17]             72
[29]             51
[30]             10
[5]               4
[32, 3]           1
[20, 30]          1
Name: count, dtype: int64

In [15]:
df['correction'].value_counts()

correction
0     5603141
8        1465
10       1465
1        1025
12       1025
7          81
11         81
Name: count, dtype: int64

In [18]:
time_intervals = pa_compute.floor_temporal(trades.column("sip_timestamp").combine_chunks(), unit="hour")
time_intervals

[
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  2003-09-10 13:00:00.000000000Z,
  ...
  2003-09-10 19:00:00.000000000Z,
  2003-09-10 19:00:00.000000000Z,
  2003-09-10 19:00:00.000000000Z,
  2003-09-10 19:00:00.000000000Z,
  2003-09-10 19:00:00.000000000Z,
  2003-09-10 20:00:00.000000000Z,
  2003-09-10 20:00:00.000000000Z,
  2003-09-10 20:00:00.000000000Z,
  2003-09-10 21:00:00.000000000Z,
  2003-09-10 22:00:00.000000000Z
]

In [19]:
trades2 = trades.append_column("time_interval", time_intervals)
trades2

pyarrow.Table
ticker: string not null
conditions: string not null
correction: int8 not null
exchange: int8 not null
id: string not null
participant_timestamp: timestamp[ns, tz=UTC] not null
price: double not null
sequence_number: int64 not null
sip_timestamp: timestamp[ns, tz=UTC] not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: timestamp[ns, tz=UTC] not null
condition_values: list<item: uint8>
  child 0, item: uint8
time_interval: timestamp[ns, tz=UTC]
----
ticker: [["A","A","A","A","A",...,"AAPL","AAPL","AAPL","AAPL","AAPL"],["AAPL","AAPL","AAPL","AAPL","AAPL",...,"ABS","ABS","ABS","ABS","ABS"],...,["YHOO","YHOO","YHOO","YHOO","YHOO",...,"ZICA","ZICA","ZICA","ZICA","ZICA"],["ZICA","ZICA","ZICA","ZICA","ZICA",...,"ZXZZT","ZXZZT","ZXZZT","ZXZZT","ZXZZT"]]
conditions: [["","","","","",...,"","","","",""],["","","","","",...,"","","","",""],...,["","","","","",...,"","","","",""],["","","","","",...,"12","15","15","12",""]]
correction: [[0,0,0,0,0,.

In [20]:
df = trades2.to_pandas()
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,condition_values,time_interval
0,A,,0,10,,1970-01-01 00:00:00+00:00,25.40,1929946249800780,2003-09-10 13:30:49.800000+00:00,42700,1,0,1970-01-01 00:00:00+00:00,[0],2003-09-10 13:00:00+00:00
1,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251320893,2003-09-10 13:30:51.320000+00:00,500,1,0,1970-01-01 00:00:00+00:00,[0],2003-09-10 13:00:00+00:00
2,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251325765,2003-09-10 13:30:51.325000+00:00,500,1,0,1970-01-01 00:00:00+00:00,[0],2003-09-10 13:00:00+00:00
3,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251330702,2003-09-10 13:30:51.330000+00:00,500,1,0,1970-01-01 00:00:00+00:00,[0],2003-09-10 13:00:00+00:00
4,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251334042,2003-09-10 13:30:51.334000+00:00,200,1,0,1970-01-01 00:00:00+00:00,[0],2003-09-10 13:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5608278,ZXZZT,12,0,12,,1970-01-01 00:00:00+00:00,817.31,1786105,2003-09-10 20:01:00+00:00,900,3,0,1970-01-01 00:00:00+00:00,[12],2003-09-10 20:00:00+00:00
5608279,ZXZZT,15,0,12,,1970-01-01 00:00:00+00:00,565.00,1788231,2003-09-10 20:01:33+00:00,0,3,0,1970-01-01 00:00:00+00:00,[15],2003-09-10 20:00:00+00:00
5608280,ZXZZT,15,0,3,,1970-01-01 00:00:00+00:00,568.08,1791346,2003-09-10 20:03:05+00:00,0,3,0,1970-01-01 00:00:00+00:00,[15],2003-09-10 20:00:00+00:00
5608281,ZXZZT,12,0,11,,1970-01-01 00:00:00+00:00,400.00,1801735,2003-09-10 21:27:22+00:00,100,3,0,1970-01-01 00:00:00+00:00,[12],2003-09-10 21:00:00+00:00


In [21]:
df['time_interval'].value_counts()

time_interval
2003-09-10 19:00:00+00:00    1142877
2003-09-10 14:00:00+00:00     983070
2003-09-10 18:00:00+00:00     820430
2003-09-10 13:00:00+00:00     777665
2003-09-10 15:00:00+00:00     734204
2003-09-10 16:00:00+00:00     551216
2003-09-10 17:00:00+00:00     536971
2003-09-10 20:00:00+00:00      49643
2003-09-10 12:00:00+00:00       7165
2003-09-10 21:00:00+00:00       4159
2003-09-10 22:00:00+00:00        759
2003-09-10 23:00:00+00:00        123
2003-09-10 11:00:00+00:00          1
Name: count, dtype: int64

In [7]:
!pwd

/home/jovyan/work/zipline-polygon-bundle/tests


In [20]:
config.cache_dir = "/home/jovyan/data/polygon_cache"

In [21]:
conditions = zpb.load_conditions(config)
show(conditions)

Got len(conditions)=130 from Polygon list_conditions.
Loaded len(conditions)=130 from /home/jovyan/data/polygon_cache/2016-01-05_2022-12-30/conditions.parquet


In [22]:
conditions

,abbreviation,asset_class,data_types,description,exchange,id,legacy,name,sip_mapping,type,update_rules
0,None,crypto,[trade],None,NaN,0,None,Regular Trade,None,regular,None
1,None,crypto,[trade],The asset was sold at the prevailing best bid ...,NaN,1,None,Sell Side,None,buy_or_sell_side,None
2,None,crypto,[trade],The asset was bought at the prevailing best as...,NaN,2,None,Buy Side,None,buy_or_sell_side,None
3,CANC,options,[trade],Transaction previously reported (other than as...,NaN,201,None,Canceled,"{'CTA': None, 'OPRA': 'A', 'UTP': None}",sale_condition,"{'consolidated': {'updates_high_low': False, '..."
4,OSEQ,options,[trade],Transaction is being reported late and is out ...,NaN,202,None,Late and Out Of Sequence,"{'CTA': None, 'OPRA': 'B', 'UTP': None}",sale_condition,"{'consolidated': {'updates_high_low': False, '..."
...,...,...,...,...,...,...,...,...,...,...,...
125,None,stocks,"[bbo, nbbo]",None,NaN,81,None,Corrected Price Indication,"{'CTA': None, 'OPRA': None, 'UTP': 'C'}",quote_condition,None
126,None,stocks,"[bbo, nbbo]",None,NaN,82,None,SIP Generated,"{'CTA': None, 'OPRA': None, 'UTP': 'E'}",sip_generated_flag,None
127,None,stocks,"[bbo, nbbo]",None,NaN,84,None,Crossed Market,"{'CTA': 'A', 'OPRA': None, 'UTP': None}",market_condition,None
128,None,stocks,"[bbo, nbbo]",None,NaN,85,None,Locked Market,"{'CTA': 'B', 'OPRA': None, 'UTP': None}",market_condition,None


In [23]:
show(conditions[conditions["type"] == "sale_condition"])

In [24]:
conditions[conditions["type"] == "sale_condition"]["id"].value_counts()

id
201    1
5      1
21     1
20     1
19     1
      ..
240    1
239    1
238    1
237    1
55     1
Name: count, Length: 73, dtype: int64

In [25]:
conditions[conditions["type"] == "sale_condition"]["legacy"].value_counts()

legacy
True    6
Name: count, dtype: int64